In [1]:
!pip install datasets

In [2]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
import wandb
user_secrets = UserSecretsClient()
secret_value = user_secrets.get_secret("HF_KEY")
login(secret_value)
secret_value1 = user_secrets.get_secret("WANDB_KEY")
wandb.login(key=secret_value1)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
from datasets import load_dataset
training_data = load_dataset("ethansimrm/patTR_biomed_raw", split = "train") 

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/ethansimrm--patTR_biomed_raw-f054c0046b36a4dc/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.


In [4]:
#print(len(training_data)) #2191537 sentence pairs prior to pre-processing
#training_data['text'][1:10] 
#This is formatted slightly differently, so let's account for that before preprocessing
#Format is TGT [TAB] SRC [TAB] medical_corpus [TAB] PatTR_Medical

In [5]:
from datasets import Dataset
source = []
target = []
for example in training_data['text']:
    example = example.strip()
    sentences = example.split("\t")
    source.append(sentences[1].strip()) #Sentences sometimes come with trailing spaces
    target.append(sentences[0].strip())
ready_data = Dataset.from_dict({"en":source, "fr":target})

In [6]:
#Seems alright.
ready_data[0]

{'en': 'SEQUENCES INVOLVED IN THE PHENOMENA OF TUMOUR SUPPRESSION, TUMOUR REVERSION, APOPTOSIS AND/OR RESISTANCE TO VIRUSES, AND THE USE OF THE SAME AS MEDICAMENTS',
 'fr': 'SEQUENCES IMPLIQUEES DANS LES PHENOMENES DE SUPPRESSION TUMORALE, REVERSION TUMORALE, APOPTOSE ET/OU RESISTANCE AUX VIRUS ET LEUR UTILISATION COMME MEDICAMENTS'}

In [7]:
import pandas as pd
#Let's start preprocessing according to Wu et al.(2022).
#First, remove duplicate sentences. 
#We can't do this in datasets, so we must convert it to a pandas DataFrame.
tempDataset = pd.DataFrame(ready_data)

In [8]:
#tempDataset[tempDataset.duplicated()] 
#Wow, there are a lot of duplicate rows, 46606 in total. 

In [9]:
tempDataset_dedup = tempDataset.drop_duplicates()

In [10]:
#Declaring helper functions separately
import langid, string
from tqdm import tqdm
langid.set_languages(["en", "fr"]) #Constrain language set
tqdm.pandas() #So I can see progress
count = lambda l1,l2: sum([1 for x in l1 if x in l2])

In [11]:
#Having verified that we can convert this back into a Dataset object, we'll just 
#do everything else in a pandas DataFrame. There are seven per-sentence checks
#we can do.

def retain_sentence(row): #This defines a good sentence we wish to retain
    word_counts = []
    #Exclude repeats and empty source or target sentences
    if ((row["en"] == row["fr"]) or (row["en"] == "") or (row["fr"] == "")):
        return False
    for sentence in (row["en"], row["fr"]):
        #Exclude sentences with mismatched () or ""
        if ((sentence.count("(") != sentence.count(")")) or 
            (sentence.count('"') % 2 != 0)):
            return False
        sent_length = len(sentence)
        #Exclude sentences with punctuation percentage > 0.4
        if (count(sentence,set(string.punctuation)) > 0.4 * sent_length):
            return False
        #Exclude sentences with > 150 words
        num_words = len(sentence.split(" "))
        if (num_words > 150):
            return False
        word_counts.append(num_words)
        #Exclude sentences with char-to-word ratio > 12 or < 1.5
        c2w_ratio = sent_length / num_words
        if ((c2w_ratio > 12) or (c2w_ratio < 1.5)):
            return False
    #Heuristic "alignment" filtering
    word_ratio = word_counts[0] / word_counts[1]
    if((word_ratio >= 9) or (1/word_ratio >= 9)):
        return False
    #Expensive language-determining step
    if ((langid.classify(row['en'])[0] != "en") or 
    (langid.classify(row['fr'])[0] != "fr")):
        return False
    return True
"""
We will skip full alignment-checking, due to infeasibility and time constraints.
SOTA-used options such as fast-align and eflomal require a large parallel training 
corpus, which we lack. AWESOME-align cannot run, even from the command line, 
due to an issue with the multiprocessing module and pickling. We can take another
crack at this in the future.

However, we can filter out obviously bad alignments by looking at word ratio;
we filter out word ratios greater than 9 or smaller than 1/9.
This (heuristic) method comes from Del et al. (2021).
"""
print()

In [12]:
tempDataset_nearlyAligned = tempDataset_dedup.progress_apply(retain_sentence, axis=1)

100%|██████████| 2144926/2144926 [38:04<00:00, 938.86it/s]


In [13]:
filtered_set = tempDataset_dedup[tempDataset_nearlyAligned]

In [14]:
filtered_set = filtered_set.reset_index(drop=True)

In [15]:
filtered_set = filtered_set.sample(frac=1).reset_index(drop=True) #Shuffle to avoid overfitting 

In [16]:
f = open("patTR_filtered.txt", "w", encoding = "utf-8")
source = filtered_set["en"]
target = filtered_set["fr"]
for i in range(len(filtered_set)):
    f.write(source[i] + "\t" + target[i] + "\n")
f.close()